In [ ]:
# from model.vgg_model import *

In [ ]:
runConfusion("vgg16-model-35-progress-report.hdf5", "Confusion Matrix VGG16 Based Model")

In [ ]:
def runTestFinalReport(model_path):
    project_root = os.getcwd()
    if os.path.exists(model_path):
        model = load_model(model_path)

    data_dir = os.path.join(project_root, 'data', 'data2.pickle')
    input_data = process_data(data_dir)
    num_samples, x_input, y_input = getDataByType(input_data, 'test')
    
    match_cnt = 0
    mismatched_inputs = []
    mismatched_labels = [] #(expected, predicted)
    for i in range(num_samples):
        if i % 1000 == 0:
            print("Testing sample %d out of %d" % (i, num_samples))
        x = x_input[i:i+1]
        y = y_input[i:i+1]

        score = model.predict(x)
        prediction = np.argmax(score)
        
        if y[0] == prediction:
            match_cnt += 1
        else:
            mismatched_inputs.append(x[0])
            mismatched_labels.append((y[0], prediction))
    
    for i in range(4):
        plt.imshow(mismatched_inputs[i])
        plt.axis('off')
        plt.gcf().set_size_inches(15, 15)
        plt.show()
        plt.close()
        print(mismatched_labels[:i])
        

In [ ]:
project_root = os.getcwd()
model_dir = os.path.join(project_root, 'checkpoints')
model_path = os.path.join(model_dir, 'vgg16-model-35-progress-report.hdf5')
runTestFinalReport(model_path)

In [1]:
from model.cnn_model import *

/home/ken/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ken/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ken/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ken/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: 

In [2]:
def TrainModelFinalReport(model, dataset='train', checkpoint_name_prefix = 'model', from_epoch=0, use_validation=True, use_augmentation=False, use_limited_augmentation=False):
    project_root = os.getcwd()
    
    checkpoint_name = checkpoint_name_prefix + '-{epoch:02d}.hdf5'

    checkpoints_dir = os.path.join(project_root, 'checkpoints', checkpoint_name)
    checkpoint = ModelCheckpoint(checkpoints_dir, monitor='val_acc', verbose=1, period=5)

    tensorboard_dir = os.path.join(project_root, 'logs')
    tensorboard = TensorBoard(tensorboard_dir)

    if from_epoch > 0:
        model_path = os.path.join(project_root, 'checkpoints', checkpoint_name.format(epoch=from_epoch))
        model = load_model(model_path, custom_objects={'tf': tf})
        initial_epoch = from_epoch
    else:
        # annealer = LearningRateScheduler(lambda x: 1e-3 * 0.95 ** (x + epochs))
        # sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[metrics.categorical_accuracy])
        initial_epoch = 0

    # ------------------------------------------------------------------------------------------------------- #

    image_dir = os.path.join(project_root, 'data', dataset)
    augmt_dir = os.path.join(project_root, 'data', '{}_aug_limited'.format(dataset) if use_limited_augmentation else '{}_aug'.format(dataset))

    num_samples = len(list(os.listdir(image_dir)))
    if use_augmentation:
        num_samples += len(list(os.listdir(augmt_dir)))
    
    # annealer = LearningRateScheduler(lambda x: 1e-3 * 0.95 ** (x + epochs))

    if use_validation:
        x_input_val, y_input_val = getValidationSetRaw(os.path.join(project_root, 'data', 'valid'))

        h = model.fit_generator(
            trainDataGeneratorRaw(image_dir, augmt_dir, BATCH_SIZE, use_augmentation),
            steps_per_epoch=np.ceil(num_samples / BATCH_SIZE),
            initial_epoch=initial_epoch,
            validation_data=(x_input_val, y_input_val),
            epochs=TOTAL_EPOCH,
            verbose=1,
            callbacks=[checkpoint, tensorboard])

    else:
        h = model.fit_generator(
            trainDataGeneratorRaw(image_dir, augmt_dir, BATCH_SIZE, use_augmentation),
            steps_per_epoch=np.ceil(num_samples / BATCH_SIZE),
            initial_epoch=initial_epoch,
            epochs=TOTAL_EPOCH,
            verbose=1,
            callbacks=[checkpoint, tensorboard])

In [3]:
def createImageModelFromVGG16FinalReport(weights = 'imagenet', pre_trained_model_freeze=None, dense_layers=True, dense_unit=4096):
    # start with a standard vgg network
    pre_trained_model = vgg16.VGG16(weights=weights, include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
    if pre_trained_model_freeze is not None:
        print(pre_trained_model.layers[pre_trained_model_freeze])
        for layer in pre_trained_model.layers[:(pre_trained_model_freeze+1)]:
            layer.trainable = False
    x = pre_trained_model.layers[-1].output
    x = Flatten(name='flatten')(x)
    if dense_layers:
        x = Dense(dense_unit, activation='relu', name='fc1')(x)
        x = Dense(dense_unit, activation='relu', name='fc2')(x)
    model_output = Dense(43, activation='softmax')(x)
    img_model = Model(inputs=pre_trained_model.input, outputs=model_output, name="img_model")
    print(img_model.summary())
    return img_model

In [4]:
# Default parameters. Using imagenet weights and no freeze.
model = createImageModelFromVGG16FinalReport()
TrainModelFinalReport(model, checkpoint_name_prefix='vgg16-final-report-imagenet-no-freeze')


Model: "img_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 96, 96, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 96, 96, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 96, 96, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 48, 48, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 48, 48, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 48, 48, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 24, 24, 128)       0

Epoch 21/30
250/250 [==============================] - 53s 211ms/step - loss: 3.5001 - categorical_accuracy: 0.0551 - val_loss: 3.4377 - val_categorical_accuracy: 0.0617
Epoch 24/30
250/250 [==============================] - 53s 211ms/step - loss: 3.5000 - categorical_accuracy: 0.0552 - val_loss: 3.4377 - val_categorical_accuracy: 0.0617
Epoch 25/30
250/250 [==============================] - 53s 211ms/step - loss: 3.5000 - categorical_accuracy: 0.0553 - val_loss: 3.4376 - val_categorical_accuracy: 0.0617

Epoch 00025: saving model to /home/ken/workspace/cs221-project/checkpoints/vgg16-final-report-imagenet-no-freeze-25.hdf5
Epoch 26/30
250/250 [==============================] - 53s 211ms/step - loss: 3.5000 - categorical_accuracy: 0.0554 - val_loss: 3.4374 - val_categorical_accuracy: 0.0617
Epoch 27/30
250/250 [==============================] - 53s 211ms/step - loss: 3.4999 - categorical_accuracy: 0.0554 - val_loss: 3.4373 - val_categorical_accuracy: 0.0617
Epoch 28/30
250/250 [=======

In [5]:
for i in [18, 14, 10, 6]:
    """
    6 - block 1 and block 2
    10 - block 1 to 3
    14 - block 1 to 4
    18 - block 1 to 5
    """
    model = createImageModelFromVGG16FinalReport(pre_trained_model_freeze=i)
    checkpoint_name_prefix = 'vgg16-final-report-imagenet-freeze-' + str(i)
    TrainModelFinalReport(model, checkpoint_name_prefix=checkpoint_name_prefix)

Model: "img_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 96, 96, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 96, 96, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 96, 96, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 48, 48, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 48, 48, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 48, 48, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 24, 24, 128)       0 


Epoch 00025: saving model to /home/ken/workspace/cs221-project/checkpoints/vgg16-final-report-imagenet-freeze-18-25.hdf5
Epoch 26/30
250/250 [==============================] - 43s 171ms/step - loss: 0.0545 - categorical_accuracy: 0.9925 - val_loss: 7.0132 - val_categorical_accuracy: 0.6567
Epoch 27/30
250/250 [==============================] - 43s 173ms/step - loss: 0.0196 - categorical_accuracy: 0.9963 - val_loss: 5.8043 - val_categorical_accuracy: 0.6752
Epoch 28/30
250/250 [==============================] - 43s 173ms/step - loss: 0.0259 - categorical_accuracy: 0.9958 - val_loss: 5.4099 - val_categorical_accuracy: 0.6730
Epoch 29/30
250/250 [==============================] - 43s 173ms/step - loss: 0.0599 - categorical_accuracy: 0.9945 - val_loss: 7.1418 - val_categorical_accuracy: 0.6617
Epoch 30/30
250/250 [==============================] - 43s 173ms/step - loss: 0.0438 - categorical_accuracy: 0.9940 - val_loss: 6.6995 - val_categorical_accuracy: 0.6653

Epoch 00030: saving model t

RuntimeError: Unable to flush file's cached information (file write failed: time = Thu Dec  5 22:44:26 2019
, filename = '/home/ken/workspace/cs221-project/checkpoints/vgg16-final-report-imagenet-freeze-10-10.hdf5', file descriptor = 69, errno = 28, error message = 'No space left on device', buf = 0x55b1db3edb00, total write size = 10456, bytes this sub-write = 10456, bytes actually written = 18446744073709551615, offset = 4096)

In [7]:
# Default parameters. Using random weights and no freeze.
model = createImageModelFromVGG16FinalReport(weights=None)
TrainModelFinalReport(model, checkpoint_name_prefix='vgg16-final-report-random-no-freeze')

Model: "img_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 96, 96, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 96, 96, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 96, 96, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 48, 48, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 48, 48, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 48, 48, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 24, 24, 128)       0 

Epoch 26/30
250/250 [==============================] - 52s 206ms/step - loss: 0.0363 - categorical_accuracy: 0.9912 - val_loss: 0.2203 - val_categorical_accuracy: 0.9586
Epoch 27/30
250/250 [==============================] - 52s 206ms/step - loss: 0.0678 - categorical_accuracy: 0.9849 - val_loss: 0.3942 - val_categorical_accuracy: 0.9407
Epoch 28/30
250/250 [==============================] - 52s 206ms/step - loss: 0.0580 - categorical_accuracy: 0.9879 - val_loss: 0.4633 - val_categorical_accuracy: 0.9032
Epoch 29/30
250/250 [==============================] - 52s 206ms/step - loss: 0.1214 - categorical_accuracy: 0.9722 - val_loss: 0.3169 - val_categorical_accuracy: 0.9314
Epoch 30/30
250/250 [==============================] - 52s 206ms/step - loss: 0.0302 - categorical_accuracy: 0.9934 - val_loss: 0.2951 - val_categorical_accuracy: 0.9484

Epoch 00030: saving model to /home/ken/workspace/cs221-project/checkpoints/vgg16-final-report-random-no-freeze-30.hdf5


In [6]:
# Default parameters. Using imagenet weights and no freeze. No dense layers.
model = createImageModelFromVGG16FinalReport(dense_layers=False)
TrainModelFinalReport(model, checkpoint_name_prefix='vgg16-final-report-imagenet-no-freeze-no-dense')

Model: "img_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 96, 96, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 96, 96, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 96, 96, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 48, 48, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 48, 48, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 48, 48, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 24, 24, 128)       0 

Epoch 27/30
250/250 [==============================] - 47s 190ms/step - loss: 0.0396 - categorical_accuracy: 0.9906 - val_loss: 0.3505 - val_categorical_accuracy: 0.9410
Epoch 28/30
250/250 [==============================] - 47s 189ms/step - loss: 0.0309 - categorical_accuracy: 0.9926 - val_loss: 0.3453 - val_categorical_accuracy: 0.9460
Epoch 29/30
250/250 [==============================] - 47s 190ms/step - loss: 0.1096 - categorical_accuracy: 0.9766 - val_loss: 0.2765 - val_categorical_accuracy: 0.9487
Epoch 30/30
250/250 [==============================] - 47s 190ms/step - loss: 0.0283 - categorical_accuracy: 0.9929 - val_loss: 0.3351 - val_categorical_accuracy: 0.9573

Epoch 00030: saving model to /home/ken/workspace/cs221-project/checkpoints/vgg16-final-report-imagenet-no-freeze-no-dense-30.hdf5


In [8]:
# Default parameters. Using random weights and no freeze. No dense unit.
model = createImageModelFromVGG16FinalReport(weights=None, dense_layers=False)
TrainModelFinalReport(model, checkpoint_name_prefix='vgg16-final-report-random-no-freeze-no-dense')

Model: "img_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 96, 96, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 96, 96, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 96, 96, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 48, 48, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 48, 48, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 48, 48, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 24, 24, 128)       0 

Epoch 27/30
250/250 [==============================] - 47s 189ms/step - loss: 0.0416 - categorical_accuracy: 0.9908 - val_loss: 0.4675 - val_categorical_accuracy: 0.9481
Epoch 28/30
250/250 [==============================] - 47s 189ms/step - loss: 0.0311 - categorical_accuracy: 0.9937 - val_loss: 0.3391 - val_categorical_accuracy: 0.9491
Epoch 29/30
250/250 [==============================] - 47s 189ms/step - loss: 0.0167 - categorical_accuracy: 0.9961 - val_loss: 0.2830 - val_categorical_accuracy: 0.9621
Epoch 30/30
250/250 [==============================] - 47s 189ms/step - loss: 0.0391 - categorical_accuracy: 0.9919 - val_loss: 0.2526 - val_categorical_accuracy: 0.9550

Epoch 00030: saving model to /home/ken/workspace/cs221-project/checkpoints/vgg16-final-report-random-no-freeze-no-dense-30.hdf5


In [5]:
# Default parameters. Using imagenet weights and no freeze. Dense layers 256 units.
model = createImageModelFromVGG16FinalReport(dense_unit=256)
TrainModelFinalReport(model, checkpoint_name_prefix='vgg16-final-report-imagenet-no-freeze-256-dense')

Model: "img_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 96, 96, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 96, 96, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 96, 96, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 48, 48, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 48, 48, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 48, 48, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 24, 24, 128)       0 

250/250 [==============================] - 48s 190ms/step - loss: 0.0563 - categorical_accuracy: 0.9867 - val_loss: 0.3038 - val_categorical_accuracy: 0.9453

Epoch 00025: saving model to /home/ken/workspace/cs221-project/checkpoints/vgg16-final-report-imagenet-no-freeze-256-dense-25.hdf5
Epoch 26/30
250/250 [==============================] - 48s 190ms/step - loss: 0.0575 - categorical_accuracy: 0.9880 - val_loss: 0.2364 - val_categorical_accuracy: 0.9486
Epoch 27/30
250/250 [==============================] - 47s 190ms/step - loss: 0.1221 - categorical_accuracy: 0.9751 - val_loss: 0.2881 - val_categorical_accuracy: 0.9424
Epoch 28/30
250/250 [==============================] - 48s 190ms/step - loss: 0.0489 - categorical_accuracy: 0.9893 - val_loss: 0.2212 - val_categorical_accuracy: 0.9560
Epoch 29/30
250/250 [==============================] - 48s 190ms/step - loss: 0.0857 - categorical_accuracy: 0.9818 - val_loss: 0.1945 - val_categorical_accuracy: 0.9634
Epoch 30/30
250/250 [=========

In [10]:
for i in [18, 14, 10, 6]:
    """
    6 - block 1 and block 2
    10 - block 1 to 3
    14 - block 1 to 4
    18 - block 1 to 5
    """
    model = createImageModelFromVGG16FinalReport(pre_trained_model_freeze=i, dense_layers=False)
    checkpoint_name_prefix = 'vgg16-final-report-imagenet-freeze-no-dense-' + str(i)
    TrainModelFinalReport(model, checkpoint_name_prefix=checkpoint_name_prefix)

Model: "img_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 96, 96, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 96, 96, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 96, 96, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 48, 48, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 48, 48, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 48, 48, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 24, 24, 128)       0 

KeyboardInterrupt: 

In [18]:
# Default parameters. Using random weights and no freeze. Dense layers 256 units.
model = createImageModelFromVGG16FinalReport(weights=None, dense_unit=256)
TrainModelFinalReport(model, checkpoint_name_prefix='vgg16-final-report-random-no-freeze-256-dense')

Model: "img_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        (None, 96, 96, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 96, 96, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 96, 96, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 48, 48, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 48, 48, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 48, 48, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 24, 24, 128)       0 

KeyboardInterrupt: 

In [6]:
# Default parameters. Using random weights and no freeze. Dense layers n units.
for n in [32]:
    model = createImageModelFromVGG16FinalReport(weights='imagenet', dense_unit=n)
    TrainModelFinalReport(model, checkpoint_name_prefix='vgg16-final-report-imagenet-no-freeze-dense-' + str(n))

Model: "img_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 96, 96, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 96, 96, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 96, 96, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 48, 48, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 48, 48, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 48, 48, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 24, 24, 128)       0 

Epoch 26/30
250/250 [==============================] - 48s 190ms/step - loss: 0.0897 - categorical_accuracy: 0.9839 - val_loss: 0.2373 - val_categorical_accuracy: 0.9615
Epoch 27/30
250/250 [==============================] - 48s 191ms/step - loss: 0.2595 - categorical_accuracy: 0.9531 - val_loss: 1.0552 - val_categorical_accuracy: 0.7897
Epoch 28/30
250/250 [==============================] - 48s 190ms/step - loss: 0.1671 - categorical_accuracy: 0.9637 - val_loss: 0.2357 - val_categorical_accuracy: 0.9540
Epoch 29/30
250/250 [==============================] - 47s 190ms/step - loss: 0.0589 - categorical_accuracy: 0.9884 - val_loss: 0.3000 - val_categorical_accuracy: 0.9532
Epoch 30/30
250/250 [==============================] - 48s 190ms/step - loss: 0.0702 - categorical_accuracy: 0.9870 - val_loss: 0.3623 - val_categorical_accuracy: 0.9620

Epoch 00030: saving model to /home/ken/workspace/cs221-project/checkpoints/vgg16-final-report-imagenet-no-freeze-dense-32-30.hdf5
Model: "img_model"


KeyboardInterrupt: 

In [ ]:
# Default parameters. Using random weights and no freeze. Dense layers n units.
for n in [64, 128]:
    model = createImageModelFromVGG16FinalReport(weights='imagenet', dense_unit=n)
    TrainModelFinalReport(model, checkpoint_name_prefix='vgg16-final-report-imagenet-no-freeze-dense-' + str(n))

In [4]:
def createImageModelFromResnetFinalReport(weights = 'imagenet', pre_trained_model_freeze=None):
    # start with a standard ResNet50 network
    pre_trained_model = resnet50.ResNet50(weights=weights, include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
    if pre_trained_model_freeze is not None:
        for layer in pre_trained_model.layers:
            if layer.name == pre_trained_model_freeze:
                print(layer.name)
                break
            layer.trainable = False
    # add trainable FC layers
    x = pre_trained_model.layers[-1].output
    x = GlobalAveragePooling2D()(x)
    model_output = Dense(43, activation='softmax')(x)
    img_model = Model(inputs=pre_trained_model.input, outputs=model_output, name="img_model")
    print(img_model.summary())
    return img_model

In [17]:
# Default parameters. Using random weights and no freeze.
model = createImageModelFromResnetFinalReport(weights=None)
TrainModelFinalReport(model, checkpoint_name_prefix='resnet-final-report-random-no-freeze')

Model: "img_model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           (None, 96, 96, 3)    0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 102, 102, 3)  0           input_10[0][0]                   
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 48, 48, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 48, 48, 64)   256         conv1[0][0]                      
__________________________________________________________________________________________

Epoch 1/30
250/250 [==============================] - 59s 234ms/step - loss: 1.5918 - categorical_accuracy: 0.5839 - val_loss: 1.6842 - val_categorical_accuracy: 0.5509
Epoch 2/30
250/250 [==============================] - 52s 208ms/step - loss: 0.1506 - categorical_accuracy: 0.9556 - val_loss: 0.5562 - val_categorical_accuracy: 0.8442
Epoch 3/30
250/250 [==============================] - 52s 209ms/step - loss: 0.0482 - categorical_accuracy: 0.9855 - val_loss: 0.5992 - val_categorical_accuracy: 0.8495
Epoch 4/30
250/250 [==============================] - 52s 210ms/step - loss: 0.0432 - categorical_accuracy: 0.9875 - val_loss: 0.3573 - val_categorical_accuracy: 0.9123
Epoch 5/30
250/250 [==============================] - 52s 210ms/step - loss: 0.0367 - categorical_accuracy: 0.9893 - val_loss: 0.5094 - val_categorical_accuracy: 0.8985

Epoch 00005: saving model to /home/ken/workspace/cs221-project/checkpoints/resnet-final-report-random-no-freeze-05.hdf5
Epoch 6/30
250/250 [==============

In [5]:
for layer_name in ["res2a_branch2a", "res3a_branch2a", "res4a_branch2a", "res5a_branch2a", "global_average_pooling2d_2"]:
    model = createImageModelFromResnetFinalReport(pre_trained_model_freeze=layer_name)
    checkpoint_name_prefix = 'resnet-final-report-imagenet-freeze-no-dense-' + layer_name
    TrainModelFinalReport(model, checkpoint_name_prefix=checkpoint_name_prefix)

/home/ken/anaconda3/lib/python3.7/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


res2a_branch2a
Model: "img_model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 96, 96, 3)    0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 102, 102, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 48, 48, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 48, 48, 64)   256         conv1[0][0]                      
___________________________________________________________________________




Epoch 1/30
250/250 [==============================] - 58s 234ms/step - loss: 0.1886 - categorical_accuracy: 0.9491 - val_loss: 0.3861 - val_categorical_accuracy: 0.9012

Epoch 2/30
250/250 [==============================] - 51s 205ms/step - loss: 0.0334 - categorical_accuracy: 0.9917 - val_loss: 0.3779 - val_categorical_accuracy: 0.9130
Epoch 3/30
250/250 [==============================] - 51s 205ms/step - loss: 0.0166 - categorical_accuracy: 0.9958 - val_loss: 0.7591 - val_categorical_accuracy: 0.8597
Epoch 4/30
250/250 [==============================] - 51s 206ms/step - loss: 0.0239 - categorical_accuracy: 0.9938 - val_loss: 2.2355 - val_categorical_accuracy: 0.6316
Epoch 5/30
250/250 [==============================] - 52s 206ms/step - loss: 0.0134 - categorical_accuracy: 0.9967 - val_loss: 0.2979 - val_categorical_accuracy: 0.9281

Epoch 00005: saving model to /home/ken/workspace/cs221-project/checkpoints/resnet-final-report-imagenet-freeze-no-dense-res2a_branch2a-05.hdf5
Epoch 6

/home/ken/anaconda3/lib/python3.7/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


res3a_branch2a
Model: "img_model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 96, 96, 3)    0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 102, 102, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 48, 48, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 48, 48, 64)   256         conv1[0][0]                      
___________________________________________________________________________

...

Epoch 1/30
250/250 [==============================] - 48s 193ms/step - loss: 0.2041 - categorical_accuracy: 0.9438 - val_loss: 0.2643 - val_categorical_accuracy: 0.9359
Epoch 2/30
250/250 [==============================] - 45s 180ms/step - loss: 0.0246 - categorical_accuracy: 0.9927 - val_loss: 0.2076 - val_categorical_accuracy: 0.9587
Epoch 3/30
250/250 [==============================] - 45s 181ms/step - loss: 0.0499 - categorical_accuracy: 0.9950 - val_loss: 644.0484 - val_categorical_accuracy: 0.3177
Epoch 4/30
250/250 [==============================] - 45s 181ms/step - loss: 0.0207 - categorical_accuracy: 0.9952 - val_loss: 0.3164 - val_categorical_accuracy: 0.9313
Epoch 5/30
250/250 [==============================] - 45s 180ms/step - loss: 0.0367 - categorical_accuracy: 0.9927 - val_loss: 0.1727 - val_categorical_accuracy: 0.9550

Epoch 00005: saving model to /home/ken/workspace/cs221-project/checkpoints/resnet-final-report-imagenet-freeze-no-dense-res4a_branch2a-05.hdf5
Epoch 6/3

/home/ken/anaconda3/lib/python3.7/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


res5a_branch2a
Model: "img_model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 96, 96, 3)    0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 102, 102, 3)  0           input_4[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 48, 48, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 48, 48, 64)   256         conv1[0][0]                      
___________________________________________________________________________

Epoch 1/30
250/250 [==============================] - 47s 190ms/step - loss: 0.2852 - categorical_accuracy: 0.9151 - val_loss: 1.3567 - val_categorical_accuracy: 0.6653
Epoch 2/30
250/250 [==============================] - 45s 180ms/step - loss: 0.0303 - categorical_accuracy: 0.9912 - val_loss: 1.4466 - val_categorical_accuracy: 0.6750
Epoch 3/30
250/250 [==============================] - 45s 180ms/step - loss: 0.0205 - categorical_accuracy: 0.9939 - val_loss: 1.2871 - val_categorical_accuracy: 0.7073
Epoch 4/30
250/250 [==============================] - 45s 181ms/step - loss: 0.0102 - categorical_accuracy: 0.9969 - val_loss: 1.2758 - val_categorical_accuracy: 0.7064
Epoch 5/30
250/250 [==============================] - 45s 181ms/step - loss: 0.0087 - categorical_accuracy: 0.9977 - val_loss: 1.4119 - val_categorical_accuracy: 0.6966

Epoch 00005: saving model to /home/ken/workspace/cs221-project/checkpoints/resnet-final-report-imagenet-freeze-no-dense-res5a_branch2a-05.hdf5
Epoch 6/30


RuntimeError: Unable to flush file's cached information (file write failed: time = Fri Dec  6 22:49:32 2019
, filename = '/home/ken/workspace/cs221-project/checkpoints/resnet-final-report-imagenet-freeze-no-dense-res5a_branch2a-25.hdf5', file descriptor = 69, errno = 28, error message = 'No space left on device', buf = 0x561605dc05c0, total write size = 2048, bytes this sub-write = 2048, bytes actually written = 18446744073709551615, offset = 4096)

In [ ]:
def createDenseModelFinalReport(weights='imagenet'):
    pre_trained_model = densenet.DenseNet121(weights=weights, include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
    x = pre_trained_model.layers[-1].output
    x = GlobalAveragePooling2D()(x)
    model_output = Dense(43, activation='softmax')(x)
    img_model = Model(inputs=pre_trained_model.input, outputs=model_output, name="img_model")
    print(img_model.summary())
    return img_model

In [13]:
TOTAL_EPOCH = 30
model = createDenseModelFinalReport()
TrainModelFinalReport(model, checkpoint_name_prefix='desnet-final-report-imagenet-30-epoch')

Model: "img_model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            (None, 96, 96, 3)    0                                            
__________________________________________________________________________________________________
zero_padding2d_3 (ZeroPadding2D (None, 102, 102, 3)  0           input_8[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 48, 48, 64)   9408        zero_padding2d_3[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 48, 48, 64)   256         conv1/conv[0][0]                 
__________________________________________________________________________________________

Epoch 1/30
250/250 [==============================] - 73s 293ms/step - loss: 0.1709 - categorical_accuracy: 0.9543 - val_loss: 0.3459 - val_categorical_accuracy: 0.9137
Epoch 2/30
250/250 [==============================] - 56s 223ms/step - loss: 0.0208 - categorical_accuracy: 0.9942 - val_loss: 0.2289 - val_categorical_accuracy: 0.9403
Epoch 3/30
250/250 [==============================] - 56s 223ms/step - loss: 0.0176 - categorical_accuracy: 0.9948 - val_loss: 0.1763 - val_categorical_accuracy: 0.9492
Epoch 4/30
250/250 [==============================] - 56s 223ms/step - loss: 0.0145 - categorical_accuracy: 0.9962 - val_loss: 0.8210 - val_categorical_accuracy: 0.8513
Epoch 5/30
250/250 [==============================] - 56s 223ms/step - loss: 0.0184 - categorical_accuracy: 0.9943 - val_loss: 0.7287 - val_categorical_accuracy: 0.8674

Epoch 00005: saving model to /home/ken/workspace/cs221-project/checkpoints/desnet-final-report-imagenet-30-epoch-05.hdf5
Epoch 6/30
250/250 [=============

In [ ]:
TOTAL_EPOCH = 50
model = createDenseModelFinalReport()
TrainModelFinalReport(model, checkpoint_name_prefix='desnet-final-report-imagenet-50-epoch')
TOTAL_EPOCH = 30 # setting it back

In [ ]:
model = createDenseModelFinalReport(weights=None)
TrainModelFinalReport(model, checkpoint_name_prefix='desnet-final-report-random-30-epoch')

In [ ]:
model = createDenseModelFinalReport()
TrainModelFinalReport(model, checkpoint_name_prefix='desnet-final-report-imagenet-full-aug-30-epoch', use_augmentation=True)